# DX 704 Week 10 Project

In this project, you will implement document search within a question and answer database and assess its performance.


The full project description and a template notebook are available on GitHub: [Project 10 Materials](https://github.com/bu-cds-dx704/dx704-project-10).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Download the SQuAD-explorer Data Set

You may use the code provided below.

In [1]:
!git clone https://github.com/rajpurkar/SQuAD-explorer

Cloning into 'SQuAD-explorer'...
remote: Enumerating objects: 5563, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 5563 (delta 11), reused 17 (delta 6), pack-reused 5539 (from 1)
Receiving objects: 100% (5563/5563), 52.26 MiB | 10.12 MiB/s, done.
Resolving deltas: 100% (3563/3563), done.


In [2]:
import json

In [3]:
with open("SQuAD-explorer/dataset/train-v1.1.json") as fp:
    train_data = json.load(fp)

In [4]:
type(train_data)

dict

In [5]:
list(train_data.keys())

['data', 'version']

In [6]:
type(train_data["data"])

list

In [7]:
len(train_data["data"])

442

In [8]:
type(train_data["data"][0])

dict

In [9]:
train_data["data"][0].keys()

dict_keys(['title', 'paragraphs'])

In [10]:
train_data["data"][0]["title"]

'University_of_Notre_Dame'

In [11]:
len(train_data["data"][0]["paragraphs"])

55

In [12]:
train_data["data"][0]["paragraphs"][0]

{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'qas': [{'answers': [{'answer_start': 515,
     'text': 'Saint Bernadette Soubirous'}],
   'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
   'id': '5733be284776f41900661182'},
  {'answers': [{'answer_start': 188, 'text': 'a copper statue of Christ

In [13]:
sum(len(doc["paragraphs"]) for doc in train_data["data"])

18896

## Part 2: Restructure JSON Data for Processing

Parse the file "SQuAD-explorer/dataset/train-v1.1.json" above to produce a file "parsed.tsv" with columns document_title, paragraph_index, and paragraph_context.
The paragraph_index column should be zero-indexed, so zero for the first paragraph of each document.
Use pandas `to_csv` method to write the file since there are many quotes and other issues to handle otherwise.

In [14]:
# YOUR CHANGES HERE

import pandas as pd

# Build one row per paragraph in each document
rows = []
for doc in train_data["data"]:
    title = doc.get("title", "")
    for idx, par in enumerate(doc.get("paragraphs", [])):
        ctx = par.get("context", "")
        rows.append({
            "document_title": title,
            "paragraph_index": idx,           # 0-based within each document
            "paragraph_context": ctx
        })

parsed_df = pd.DataFrame(
    rows,
    columns=["document_title", "paragraph_index", "paragraph_context"]
)

parsed_df.to_csv("parsed.tsv", sep="\t", index=False)

print(f"Wrote {len(parsed_df):,} rows to parsed.tsv")

Wrote 18,896 rows to parsed.tsv


Submit "parsed.tsv" in Gradescope.

## Part 3: Prepare Suitable Paragraph Vectors for Document Search

Design and implement paragraph vectors based on their text with length 1024.
Note that this will be much smaller than the number of distinct words in the training data.

Hint: you can base your vectors on any techniques covered in this module so far.
Beware that they will be automatically assessed (along with the question vectors of part 4) to make sure they retain useful information.

In [15]:
# YOUR CHANGES HERE

import re, math, json, hashlib
from collections import Counter, defaultdict
import numpy as np
import pandas as pd

DIM = 1024  # vector length per instructions

def tokenize(text: str):
    # simple, robust tokenizer: lowercase & split on non-letters/digits
    return [t for t in re.split(r"[^A-Za-z0-9]+", text.lower()) if t]

def token_index(token: str, dim: int = DIM) -> int:
    # stable (process-independent) hash to index using md5
    return int(hashlib.md5(token.encode("utf-8")).hexdigest(), 16) % dim

In [16]:
paragraph_tokens = []      # list of token lists per paragraph (to avoid re-tokenizing)
df_counts = defaultdict(int)
num_paragraphs = 0

for doc in train_data["data"]:
    for par in doc.get("paragraphs", []):
        toks = tokenize(par.get("context", ""))
        paragraph_tokens.append(toks)
        num_paragraphs += 1
        for tok in set(toks):
            df_counts[tok] += 1

# Precompute IDF for tokens we observed
idf = {}
N = float(num_paragraphs)
for tok, df in df_counts.items():
    # add-1 smoothed IDF to be safe; classic log(N / (1 + df))
    idf[tok] = math.log(N / (1.0 + df))

In [ ]:
rows = []
para_idx_global = 0 

for doc in train_data["data"]:
    title = doc.get("title", "")
    for p_idx, par in enumerate(doc.get("paragraphs", [])):
        toks = paragraph_tokens[para_idx_global]
        para_idx_global += 1

        # term frequencies for this paragraph
        tf = Counter(toks)

        # build hashed TF-IDF vector
        vec = np.zeros(DIM, dtype=np.float32)
        for tok, cnt in tf.items():
            j = token_index(tok, DIM)
            # accumulate in case of collisions
            vec[j] += float(cnt) * idf.get(tok, 0.0)

        # L2 normalize for scale invariance
        norm = float(np.linalg.norm(vec))
        if norm > 0.0:
            vec /= norm

        # store as JSON-encoded list
        vec_json = json.dumps(vec.tolist())

        rows.append({
            "document_title": title,
            "paragraph_index": p_idx,               # 0-based within each document
            "paragraph_vector_json": vec_json
        })


Save your paragraph vectors in a file "paragraph-vectors.tsv.gz" with columns document_title, paragraph_index, and paragraph_vector_json where paragraph_vector_json is a JSON encoded list.

Hint: don't forget the ".gz" extension indicating gzip compression.
The Pandas `.to_csv` method will automatically add the compression if you save data with a filename ending in ".gz", so you just need to pass it the right filename.

In [18]:
# YOUR CHANGES HERE

para_vecs_df = pd.DataFrame(
    rows,
    columns=["document_title", "paragraph_index", "paragraph_vector_json"]
)

out_path = "paragraph-vectors.tsv.gz"
para_vecs_df.to_csv(out_path, sep="\t", index=False)

print(f"Built vectors for {len(para_vecs_df):,} paragraphs; saved to {out_path}")

Built vectors for 18,896 paragraphs; saved to paragraph-vectors.tsv.gz


Submit "paragraph-vectors.tsv.gz" in Gradescope.

## Part 4: Encode Question Vectors with the Same Design

Read the questions in "questions.tsv" and encode them in the same way that you encoded the paragraph vectors.

In [19]:
# YOUR CHANGES HERE

questions_df = pd.read_csv("questions.tsv", sep="\t")

rows = []
for _, row in questions_df.iterrows():
    qid = int(row["question_id"])
    qtext = row["question"]

    toks = tokenize(qtext)
    tf = Counter(toks)

    vec = np.zeros(DIM, dtype=np.float32)
    for tok, cnt in tf.items():
        j = token_index(tok, DIM)
        vec[j] += float(cnt) * idf.get(tok, 0.0)

    # L2 normalize
    norm = float(np.linalg.norm(vec))
    if norm > 0.0:
        vec /= norm

    rows.append({
        "question_id": qid,
        "question_vector_json": json.dumps(vec.tolist())
    })


Save your question vectors in "question-vectors.tsv" with columns question_id and question_vector_json.

In [20]:
# YOUR CHANGES HERE

question_vecs_df = pd.DataFrame(rows, columns=["question_id", "question_vector_json"])
question_vecs_df.to_csv("question-vectors.tsv", sep="\t", index=False)

print(f"Encoded {len(question_vecs_df):,} questions → question-vectors.tsv")

Encoded 100 questions → question-vectors.tsv


Submit "question-vectors.tsv" in Gradescope.

## Part 5: Match Questions to Paragraphs using Nearest Neighbors

Match your question vectors to paragraph vectors and identify the top 5 paragraph vectors for each question using nearest neighbors.
Specifically, use the Euclidean distance between the vectors.


In [21]:
# YOUR CHANGES HERE

para_df = pd.read_csv("paragraph-vectors.tsv.gz", sep="\t")
q_df    = pd.read_csv("question-vectors.tsv", sep="\t")

def parse_vec(col):
    return np.array([json.loads(x) for x in col], dtype=np.float32)

P = parse_vec(para_df["paragraph_vector_json"])  # shape: (num_paragraphs, 1024)
Q = parse_vec(q_df["question_vector_json"])      # shape: (num_questions, 1024)

In [22]:
# Renormalize in case anything drifted
def l2_normalize(X):
    norms = np.linalg.norm(X, axis=1, keepdims=True)
    norms[norms == 0.0] = 1.0
    return X / norms

P = l2_normalize(P)
Q = l2_normalize(Q)

In [ ]:
# d^2 = ||a - b||^2 = ||a||^2 + ||b||^2 - 2 a·b = 2 - 2 a·b
K = 5
results = []

# Batch over questions to limit memory
BATCH = 1024 if Q.shape[0] > 2048 else Q.shape[0]

for start in range(0, Q.shape[0], BATCH):
    stop = min(start + BATCH, Q.shape[0])
    Qb = Q[start:stop]                                  # (b, d)
    sims = Qb @ P.T                                     # (b, n_paragraphs), cosine similarities
    # Get top-K by similarity
    # Use argpartition for speed, then sort those K indices by true distance
    topk_idx_part = np.argpartition(-sims, K-1, axis=1)[:, :K]  # (b, K)
    # Gather sims for those indices
    topk_sims = np.take_along_axis(sims, topk_idx_part, axis=1) # (b, K)
    # Convert to Euclidean distances (on normalized vectors)
    topk_d2 = 2.0 - 2.0 * topk_sims
    # Numerical safety (clip negatives from floating error), then sqrt
    topk_dist = np.sqrt(np.maximum(topk_d2, 0.0))

    # Now sort each row's K by ascending distance
    order = np.argsort(topk_dist, axis=1)
    topk_sorted_idx = np.take_along_axis(topk_idx_part, order, axis=1)
    topk_sorted_dist = np.take_along_axis(topk_dist, order, axis=1)

    # Emit rows
    for i, qrow in enumerate(q_df.iloc[start:stop].itertuples(index=False)):
        qid = int(qrow.question_id)
        for rank in range(K):
            p_idx = int(topk_sorted_idx[i, rank])
            # dist = float(topk_sorted_dist[i, rank])

            results.append({
                "question_id": qid,
                "question_rank": rank + 1,  # 1 = best / closest
                "document_title": para_df.iloc[p_idx]["document_title"],
                "paragraph_index": int(para_df.iloc[p_idx]["paragraph_index"]),
            })

Save your top matches in a file "question-matches.tsv" with columns question_id, question_rank, document_title, and paragraph_index.


In [24]:
# YOUR CHANGES HERE

matches_df = pd.DataFrame(
    results,
    columns=["question_id", "question_rank", "document_title", "paragraph_index"]
)

matches_df.to_csv("question-matches.tsv", sep="\t", index=False)
print(f"Wrote top {K} matches for {q_df.shape[0]} questions to question-matches.tsv")

Wrote top 5 matches for 100 questions to question-matches.tsv


Submit "question-matches.tsv" in Gradescope.

## Part 6: Spot Check Question and Paragraph Matches

Review the paragraphs matched to the first 5 questions (sorted by question_id ascending).
Which paragraph was the worst match for each question?


Submit "worst-paragraphs.tsv" in Gradescope.

Write a file "worst-paragraphs.tsv" with three columns question_id, document_title, paragraph_index.

In [25]:
# Load matches produced in Part 5
matches_df = pd.read_csv("question-matches.tsv", sep="\t")

# Find the first 5 question_ids (ascending)
first_five_qids = sorted(matches_df["question_id"].unique())[:5]

# Select the worst-ranked (rank=5) paragraph per question
worst_rows = (
    matches_df[matches_df["question_id"].isin(first_five_qids)]
    .query("question_rank == 5")
    .loc[:, ["question_id", "document_title", "paragraph_index"]]
    .sort_values(["question_id"])
    .reset_index(drop=True)
)

# Save required file
worst_rows.to_csv("worst-paragraphs.tsv", sep="\t", index=False)
print("Saved worst-paragraphs.tsv with", len(worst_rows), "rows")

Saved worst-paragraphs.tsv with 5 rows


## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 8: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.